In [1]:
import dgl
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
import torch


# Define a Heterograph Conv model

class RGCN(nn.Module):
    def __init__(self, emb_types, emb_size, hid_feats, out_feats, rel_names):
        super().__init__()
        # https://www.jianshu.com/p/767950b560c4
        embed_dict = {ntype : nn.Parameter(torch.Tensor(emb_types[ntype], emb_size))
                      for ntype in emb_types.keys()}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(emb_size, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph):
        # inputs are features of nodes
        h = self.conv1(graph, self.embed)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [2]:
g = dgl.load_graphs("./graphs/industrial_and_scientific_5_core.dgl")[0][0]
g

Graph(num_nodes={'Brand': 1900, 'Customer': 11041, 'Product': 5334, 'Review': 77071},
      num_edges={('Brand', 'rev_SOLD_BY', 'Product'): 5555, ('Customer', 'WROTE', 'Review'): 77071, ('Product', 'SOLD_BY', 'Brand'): 5555, ('Product', 'rev_REVIEW_OF', 'Review'): 77071, ('Review', 'REVIEW_OF', 'Product'): 77071, ('Review', 'rev_WROTE', 'Customer'): 77071},
      metagraph=[('Brand', 'Product', 'rev_SOLD_BY'), ('Product', 'Brand', 'SOLD_BY'), ('Product', 'Review', 'rev_REVIEW_OF'), ('Customer', 'Review', 'WROTE'), ('Review', 'Product', 'REVIEW_OF'), ('Review', 'Customer', 'rev_WROTE')])

In [3]:
model = RGCN({x: g.number_of_nodes(x) for x in g.ntypes}, 512, 256, 2, g.etypes)

labels = g.nodes["Review"].data["Positive"]
train_mask = g.nodes["Review"].data["train_mask"]
test_mask = g.nodes["Review"].data["test_mask"]

In [4]:
opt = torch.optim.Adam(model.parameters())

for epoch in range(250):
    model.train()

    logits = model(g)["Review"]
    loss = F.cross_entropy(logits[train_mask == 1], labels[train_mask == 1].type(torch.long))
    opt.zero_grad()
    loss.backward()
    opt.step()
    print("Epoch:", epoch, "Loss:", loss.item())

/opt/anaconda3/envs/huggingface/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch: 0 Loss: 0.6928762793540955
Epoch: 1 Loss: 0.6704129576683044
Epoch: 2 Loss: 0.64774489402771
Epoch: 3 Loss: 0.621573805809021
Epoch: 4 Loss: 0.5905534625053406
Epoch: 5 Loss: 0.5541093349456787
Epoch: 6 Loss: 0.5124903917312622
Epoch: 7 Loss: 0.4669405519962311
Epoch: 8 Loss: 0.41965290904045105
Epoch: 9 Loss: 0.37351658940315247
Epoch: 10 Loss: 0.3315752148628235
Epoch: 11 Loss: 0.29633381962776184
Epoch: 12 Loss: 0.26914796233177185
Epoch: 13 Loss: 0.24983493983745575
Epoch: 14 Loss: 0.2367929071187973
Epoch: 15 Loss: 0.22761522233486176
Epoch: 16 Loss: 0.21990403532981873
Epoch: 17 Loss: 0.21187925338745117
Epoch: 18 Loss: 0.20263557136058807
Epoch: 19 Loss: 0.19210666418075562
Epoch: 20 Loss: 0.18078264594078064
Epoch: 21 Loss: 0.16943512856960297
Epoch: 22 Loss: 0.1588880568742752
Epoch: 23 Loss: 0.1498504877090454
Epoch: 24 Loss: 0.1428607851266861
Epoch: 25 Loss: 0.13811862468719482
Epoch: 26 Loss: 0.13531246781349182
Epoch: 27 Loss: 0.1337178349494934
Epoch: 28 Loss: 0.1

KeyboardInterrupt: 

In [5]:
preds = model(g)["Review"][test_mask == 1].detach().numpy()
y_test = labels[test_mask == 1]

In [6]:
from sklearn.metrics import classification_report

print(classification_report(y_test, preds.argmax(1)))

              precision    recall  f1-score   support

         0.0       0.24      0.24      0.24      4750
         1.0       0.89      0.89      0.89     33786

    accuracy                           0.81     38536
   macro avg       0.56      0.56      0.56     38536
weighted avg       0.81      0.81      0.81     38536



In [7]:
from sklearn.metrics import f1_score
f1_score(y_test, preds.argmax(1))

0.892437472250999